In [1]:
!pip install bokeh

In [3]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

In [4]:
# Upload your dataset first
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))

Saving dataset.csv to dataset (1).csv


In [5]:
# Combine date and time into a timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [6]:
# Model 1 Pricing Logic
df['Price'] = np.nan
base_price = 10
alpha = 2.5
min_price = 5
max_price = 30
lot_groups = df.groupby('SystemCodeNumber')
result_frames = []
for lot_id, group in lot_groups:
    group = group.sort_values('Timestamp')
    prices = [base_price]
    for i in range(1, len(group)):
        occupancy = group.iloc[i]['Occupancy']
        capacity = group.iloc[i]['Capacity']
        prev_price = prices[-1]
        new_price = prev_price + alpha * (occupancy / capacity)
        new_price = round(min(max(new_price, min_price), max_price), 2)
        prices.append(new_price)
    group['Price'] = prices
    result_frames.append(group)
final_df = pd.concat(result_frames).reset_index(drop=True)
final_df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,Price
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,10.00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,10.28
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,10.63
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,11.09
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,11.74
